## 3.2 Gensim进行LDA

In [8]:
##########################1. 加入自定义词频,使其不易分割##########################
import jieba

jieba.suggest_freq('沙瑞金',True)
jieba.suggest_freq('易学习',True)
jieba.suggest_freq('王大路',True)
jieba.suggest_freq('京州', True)

##########################2. 对文档分词并将结果写入文件##########################
# 第一个文档分词
with open('../自然语言处理/data/nlp_test0.txt') as f :
    doc1 = f.read()
    doc_decode = doc1.decode('utf-8') # utf-8
    doc_cut = jieba.cut(doc_decode)
    res = ' '.join(doc_cut).encode('utf-8')
    with open('../自然语言处理/data/nlp_test1.txt','w') as f2:
        f2.write(res)
# 第二个文档分词 
with open('../自然语言处理/data/nlp_test2.txt') as f :
    doc1 = f.read()
    doc_decode = doc1.decode('utf-8') # utf-8
    doc_cut = jieba.cut(doc_decode)
    res = ' '.join(doc_cut).encode('utf-8')
    with open('../自然语言处理/data/nlp_test3.txt','w') as f2:
        f2.write(res)
# 第三个文档分词
with open('../自然语言处理/data/nlp_test4.txt') as f :
    doc1 = f.read()
    doc_decode = doc1.decode('utf-8') # utf-8
    doc_cut = jieba.cut(doc_decode)
    res = ' '.join(doc_cut).encode('utf-8')
    with open('../自然语言处理/data/nlp_test5.txt','w') as f2:
        f2.write(res)
        

In [10]:
##########################4. 导入停用词表 ##########################
stpword_path = '../自然语言处理/data/stop_words.txt'
with open(stpword_path,'rb') as f:
    stw_list = f.read().splitlines()
    
##########################3. 读入分好词的数据文件到内存 ##########################
with open('../自然语言处理/data/nlp_test1.txt') as f3:
    res1  = f3.read()
    for w in f3.read().split(' '): 
        if w not in stw_list: 
            1
    print res1
with open('../自然语言处理/data/nlp_test3.txt') as f4:
    res2 = f4.read()
print res2
with open('../自然语言处理/data/nlp_test5.txt') as f5:
    res3 = f5.read()
print res3

沙瑞金 赞叹 易学习 的 胸怀 ， 是 金山 的 百姓 有福 ， 可是 这件 事对 李达康 的 触动 很大 。 易学习 又 回忆起 他们 三人 分开 的 前一晚 ， 大家 一起 喝酒 话别 ， 易学习 被 降职 到 道口 县当 县长 ， 王大路 下海经商 ， 李达康 连连 赔礼道歉 ， 觉得 对不起 大家 ， 他 最 对不起 的 是 王大路 ， 就 和 易学习 一起 给 王大路 凑 了 5 万块 钱 ， 王大路 自己 东挪西撮 了 5 万块 ， 开始 下海经商 。 没想到 后来 王大路 竟然 做 得 风生水 起 。 沙瑞金 觉得 他们 三人 ， 在 困难 时期 还 能 以沫 相助 ， 很 不 容易 。 

沙瑞金 向 毛娅 打听 他们 家 在 京州 的 别墅 ， 毛娅 笑 着 说 ， 王大路 事业有成 之后 ， 要 给 欧阳 菁 和 她 公司 的 股权 ， 她们 没有 要 ， 王大路 就 在 京州 帝豪园 买 了 三套 别墅 ， 可是 李达康 和 易学习 都 不要 ， 这些 房子 都 在 王大路 的 名下 ， 欧阳 菁 好像 去 住 过 ， 毛娅 不想 去 ， 她 觉得 房子 太大 很 浪费 ， 自己 家住 得 就 很 踏实 。 

347 年 （ 永和 三年 ） 三月 ， 桓 温兵 至 彭模 （ 今 四川 彭山 东南 ） ， 留下 参军 周楚 、 孙盛 看守 辎重 ， 自己 亲率 步兵 直攻 成都 。 同月 ， 成汉 将领 李福 袭击 彭模 ， 结果 被 孙盛 等 人 击退 ； 而桓温 三 战三胜 ， 一直 逼近 成都 。 

